# Homework 2: A/B Hypothesis Testing and Regression

## Due Tuesday, May 29, start of class

### Jake Pitkin

## Part 1 : A/B Hypothesis Testing

Null hypothesis: Ad A and ad B are the same. That is a_prob - b_prob = 0.

In [99]:
import scipy as sc
from scipy.stats import norm

a_views = 1000
a_clicks = 500
a_prob = a_clicks/a_views

b_views = 1000
b_clicks = 550
b_prob = b_clicks/b_views

p_hat = (a_clicks + b_clicks)/(a_views + b_views)
q_hat = 1 - p_hat
print("p_hat:", p_hat)
print("q_hat:", q_hat)

z = (a_prob - b_prob) / sc.sqrt(p_hat*q_hat*(1/a_views + 1/b_views))
print("Z:", z)

p_value = 1 - norm.cdf(z)
print("p-value:", p_value)

if p_value < 0.01 or p_value > 0.99:
    print("Null hypothesis rejected with 1% significance level.")
if p_value < 0.05 or p_value > 0.95:
    print("Null hypothesis rejected with 5% significance level.")

p_hat: 0.525
q_hat: 0.475
Z: -2.2388683142
p-value: 0.98741775715
Null hypothesis rejected with 5% significance level.


## Part 2: Regression of real estate data

### Task 1: Import the data

In [34]:
import pandas as pd

real_estate_one = pd.read_csv("realEstate1.csv", sep=',')
real_estate_two = pd.read_csv("realEstate2.csv", sep=',')
df = pd.concat([real_estate_one, real_estate_two], ignore_index=True)
df.columns.values

array(['ListNo', 'Access', 'AdditInfo', 'Acres', 'ListAgt', 'AirType',
       'Amenities', 'Area', 'BackDim', 'Basemnt', 'BsmntFin', 'City',
       'ListCoAgt', 'CompBac', 'CompDays', 'CompSac', 'ContPh1', 'ContPh2',
       'Contact', 'ContractDt', 'County', 'CrprtCap', 'DVR', 'DaysOnMkt',
       'Deck', 'DirPre', 'DirPost', 'EWCoord', 'EntrdBy', 'EntryDt',
       'Excl', 'ExpDtDisp', 'Ext', 'FeatExt', 'FeatInt', 'Floor',
       'Frontage', 'GaragCap', 'Garage', 'HOAFee', 'HOATerms', 'Heat',
       'HouseNbr', 'Image', 'Incl', 'Irregular', 'LandScape', 'Latitude',
       'ListDt', 'ListType', 'Longitude', 'Lot', 'LstPrice', 'NSCoord',
       'NumDish', 'NumDispose', 'NumOvRng', 'NumRefg', 'OffMktDt',
       'ListOff', 'OpenHseDt', 'Owner', 'P1Bed', 'P1BthFull', 'P1BthHalf',
       'P1BthTq', 'P1Fire', 'P1Rent', 'P1Sqf', 'P1FamDen', 'P1Kitch',
       'P1Bar', 'P1Formal', 'P1SemiForm', 'P1Laundry', 'P2Bed',
       'P2BthFull', 'P2BthHalf', 'P2BthTq', 'P2Fire', 'P2Rent', 'P2Sqf',
       '

### Task 2: Clean the data

In [36]:
# Remove houses that don't have a list price between $200,000 and $1,000,000
df = df.drop(df[(df['LstPrice'] <= 200000) | (df['LstPrice'] >= 1000000)].index)

# Only keep attributes we are interested in
df = df[['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 
         'PkgSpacs', 'PropType', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']].copy()



(260, 15)
